In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown, Output, interact_manual
from IPython.display import display, FileLink
from collections import Counter
import zipfile, os
# -------------------------
# Scraping Functions
# -------------------------

def scrape_weather():
    cities = ["London", "New York", "Tokyo", "Delhi", "Sydney"]
    data = []
    for city in cities:
        url = f"https://wttr.in/{city}?format=%t"
        try:
            response = requests.get(url, timeout=5)
            temp = response.text.strip()
            data.append({"City": city, "Temperature": temp})
        except:
            data.append({"City": city, "Temperature": "N/A"})
    return pd.DataFrame(data)

def scrape_news(topic="General"):
    urls = {
        "General": "https://www.bbc.com/news",
        "Sports": "https://www.bbc.com/sport",
        "Environment": "https://www.bbc.com/news/science_and_environment",
        "Social Trends": "https://www.bbc.com/news/world",
    }
    url = urls.get(topic, urls["General"])
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    headlines = [h.get_text().strip() for h in soup.select("h3") if len(h.get_text().strip()) > 10]
    return pd.DataFrame(headlines[:15], columns=["Headline"])

def scrape_crypto():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {"vs_currency": "usd", "order": "market_cap_desc", "per_page": 5, "page": 1}
    response = requests.get(url, params=params)
    data = response.json()
    return pd.DataFrame([{"Name": c["name"], "Price (USD)": c["current_price"]} for c in data])

# -------------------------
# Visualization Functions
# -------------------------

def plot_weather(df):
    df["Temperature"] = df["Temperature"].str.replace("+","").str.replace("°C","").str.replace("°F","")
    df["Temperature"] = pd.to_numeric(df["Temperature"], errors="coerce")
    df.plot(x="City", y="Temperature", kind="bar", legend=False, title="City Temperatures")
    plt.ylabel("°C")
    plt.show()

def plot_news(df):
    words = " ".join(df["Headline"]).split()
    common = Counter(words).most_common(10)
    if not common:
        print("No words to plot.")
        return
    freq_df = pd.DataFrame(common, columns=["Word", "Frequency"])
    freq_df.plot(x="Word", y="Frequency", kind="bar", legend=False, title="Top Words in Headlines")
    plt.ylabel("Count")
    plt.show()

def plot_crypto(df):
    df.plot(x="Name", y="Price (USD)", kind="bar", legend=False, title="Top 5 Cryptos (USD)")
    plt.ylabel("Price (USD)")
    plt.show()

# -------------------------
# Dashboard
# -------------------------

output = Output()

def dashboard(option):
    output.clear_output()
    with output:
        if option == "Weather":
            df = scrape_weather()
            filename = "weather.csv"
            df.to_csv(filename, index=False)
            display(df)
            plot_weather(df)
            display(FileLink(filename, result_html_prefix="⬇️ Click to download: "))
            
        elif option == "News":
            def show_news(topic):
                df = scrape_news(topic)
                filename = f"news_{topic.lower().replace(' ','_')}.csv"
                df.to_csv(filename, index=False)
                display(df)
                plot_news(df)
                display(FileLink(filename, result_html_prefix="⬇️ Click to download: "))
            
            topic_dropdown = Dropdown(options=["General", "Sports", "Environment", "Social Trends"], description="Topic:")
            interact_manual(show_news, topic=topic_dropdown)
            
        elif option == "Cryptocurrency":
            df = scrape_crypto()
            filename = "crypto.csv"
            df.to_csv(filename, index=False)
            display(df)
            plot_crypto(df)
            display(FileLink(filename, result_html_prefix="⬇️ Click to download: "))
        
        elif option == "All Data":
            # Scrape all
            weather_df = scrape_weather()
            news_df = scrape_news("General")
            crypto_df = scrape_crypto()
            
            # Save CSVs
            weather_file = "weather.csv"
            news_file = "news.csv"
            crypto_file = "crypto.csv"
            weather_df.to_csv(weather_file, index=False)
            news_df.to_csv(news_file, index=False)
            crypto_df.to_csv(crypto_file, index=False)
          
# -------------------------
# Interactive Menu
# -------------------------

dropdown = Dropdown(options=["Weather", "News", "Cryptocurrency", "All Data"], description="Choose:")
interact(dashboard, option=dropdown)
display(output)